# 0 Imports and Visual Func

In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype

In [2]:
def jupyter_settings():

    # change display settings
    pd.options.display.max_columns = None
    pd.options.display.max_rows = 50
	# supress scientific notation
    pd.set_option('display.expand_frame_repr', False)
    pd.set_option('display.float_format',  '{:,.2f}'.format)

jupyter_settings();

In [3]:
# variables month and year
month = 3
year = 2023
day = 10
day_demissao = 20

## 0.1 Load Files

In [4]:
df = pd.read_excel("/mnt/c/Users/luizl/repos/spar/pdi_rv_project/files/hc_geral_adm.xlsx", engine="openpyxl", sheet_name=0)

In [5]:
df_cargos = pd.read_excel("/mnt/c/Users/luizl/repos/spar/pdi_rv_project/files/cargos_de_para.xlsx", engine="openpyxl", sheet_name=0)

In [6]:
df_tempo = pd.read_excel("/mnt/c/Users/luizl/repos/spar/pdi_rv_project/files/hc_geral_adm.xlsx", engine="openpyxl", sheet_name=1)

# 1 Open Files and Data Cleaning

## 1.1 - Limpando o HC

In [7]:
# Create a column ultima data de demissão
df['demissao'] = df.apply(lambda x : x['DATA DESLIGAMENTO CLT'] if x['STATUS'] == 'DESLIGADO' and pd.notnull(x['DATA DESLIGAMENTO CLT']) else 
                            x['DATA DESLIGAMENTO TEMP.'] if x['STATUS'] == 'DESLIGADO' and pd.notnull(x['DATA DESLIGAMENTO TEMP.']) else
                            np.nan, axis=1)

# remove unused columns
df = df.iloc[:,[0, 1, 4, 9, 19, 20, 21, 22, 25, 29, 32, -1]]

# remove - and . from CPF column
df['CPF'] = df['CPF'].str.replace(".", '', regex= False)
df['CPF'] = df['CPF'].str.replace("-", '', regex= False)

# remove extra regions
df = df.loc[df['MICRO REGIONAL'].notnull()]
df['MICRO REGIONAL'] = df['MICRO REGIONAL'].apply(
    lambda x : 
    x[:x.find(',')] if x.find(',') > 0 else x
)

# remove extra cargo
df['CARGO'] = df['CARGO'].apply(
    lambda x : 
    x[:x.find('/')] if x.find('/') > 0 else x
)

# trim all columns
for columns in df.columns:
    if is_string_dtype(df[columns]):
        df[columns] = df[columns].str.strip()
    else:
        df[columns] = df[columns]

df = df.merge(
    df_cargos, how='left', left_on='CARGO', right_on='CARGOS NOVOS', suffixes=(False, False)
)

# drop unused columns
df = df.drop(['CARGOS NOVOS', 'CARGO'],axis=1)


'''
Cleaning Data Set Feristas
'''
# Create a column ultima data de demissão
df_tempo['demissao'] = df_tempo.apply(lambda x : x['DATA DESLIGAMENTO CLT'] if x['STATUS'] == 'DESLIGADO' and pd.notnull(x['DATA DESLIGAMENTO CLT']) else 
                            x['DATA DESLIGAMENTO TEMP.'] if x['STATUS'] == 'DESLIGADO' and pd.notnull(x['DATA DESLIGAMENTO TEMP.']) else
                            np.nan, axis=1)

df_tempo = df_tempo.iloc[:, [0, 1, 5, 9, 19, 20, 21, 22, 25, 29, 32, -1]]

# remove - and . from CPF column
df_tempo['CPF'] = df_tempo['CPF'].str.replace(".", '', regex= True)
df_tempo['CPF'] = df_tempo['CPF'].str.replace("-", '', regex= True)

# remove extra MICRO REGIONAL
df_tempo = df_tempo.loc[df_tempo['MICRO REGIONAL'].notnull()]
df_tempo['MICRO REGIONAL'] = df_tempo['MICRO REGIONAL'].apply(
    lambda x : 
    x[:x.find(',')] if x.find(',') > 0 else x
)

# remove erros on CARGOS
df_tempo['CARGO'] = df_tempo['CARGO'].apply(
    lambda x : 
    x[:x.find('/')] if x.find('/') > 0 else x
)

# trim all columns
for columns in df_tempo.columns:
    if is_string_dtype(df_tempo[columns]):
        df_tempo[columns] = df_tempo[columns].str.strip()
    else:
        df_tempo[columns] = df_tempo[columns]

df_tempo = df_tempo.merge(
    df_cargos, how='left', left_on='CARGO', right_on='CARGOS NOVOS', suffixes=(False, False)
)

# drop unused columns
df_tempo = df_tempo.drop(['CARGOS NOVOS', 'CARGO'],axis=1)

# concat df_hc
df_hc = pd.concat([df, df_tempo])


In [8]:
df1 = df_hc.copy()
print(f'Total Linhas :  {df1.shape[0]}')
print(f'Total Colunas: {df1.shape[1]}')

Total Linhas :  1372
Total Colunas: 12


In [9]:
# remove os novos contratados
df1 = df1.loc[df1['ADMISSÃO/ARRASTAR FÓRMULA'] <= pd.to_datetime(str(year)+'-'+str(month)+'-'+str(day))]
print(f'Total Linhas :  {df1.shape[0]}')
print(f'Total Colunas: {df1.shape[1]}')

Total Linhas :  1359
Total Colunas: 12


In [16]:
df1 = df1.loc[(df1['demissao'] >= pd.to_datetime(str(year)+'-'+str(month)+'-'+str(day_demissao))) | (df1['demissao'].isnull())]
print(f'Total Linhas :  {df1.shape[0]}')
print(f'Total Colunas: {df1.shape[1]}')

Total Linhas :  642
Total Colunas: 12


In [18]:
df1.to_csv("/mnt/c/Users/luizl/repos/spar/pdi_rv_project/files/teste.csv", sep=';', encoding='utf-8-sig', index=False)